In [4]:
import pygame
import random

# Initialize pygame
pygame.init()

# Screen dimensions
WIDTH, HEIGHT = 800, 600
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Memory Matching Game")

# Colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED = (255, 0, 0)
GRAY = (200, 200, 200)
LIGHT_GRAY = (220, 220, 220)

# Font
font = pygame.font.Font(None, 50)

# Game variables
ROWS, COLS = 4, 4  # 4x4 grid
CARD_WIDTH, CARD_HEIGHT = WIDTH // COLS, HEIGHT // ROWS
cards = []
matched = []
flipped = []
running = True
flip_back_timer = 0  # Timer for flipping cards back

# Create cards
def create_cards():
    icons = [i for i in range(1, (ROWS * COLS // 2) + 1)] * 2
    random.shuffle(icons)
    for r in range(ROWS):
        row = []
        for c in range(COLS):
            card_rect = pygame.Rect(c * CARD_WIDTH, r * CARD_HEIGHT, CARD_WIDTH, CARD_HEIGHT)
            row.append({"rect": card_rect, "icon": icons.pop(), "flipped": False, "matched": False})
        cards.append(row)

# Draw the grid of cards
def draw_cards():
    for row in cards:
        for card in row:
            if card["matched"]:
                pygame.draw.rect(screen, LIGHT_GRAY, card["rect"])  # Matched cards
            elif card["flipped"]:
                pygame.draw.rect(screen, WHITE, card["rect"])
                icon_text = font.render(str(card["icon"]), True, BLACK)
                icon_rect = icon_text.get_rect(center=card["rect"].center)
                screen.blit(icon_text, icon_rect)
            else:
                pygame.draw.rect(screen, RED, card["rect"])
            pygame.draw.rect(screen, BLACK, card["rect"], 3)  # Border

# Check for match
def check_match():
    global flipped, flip_back_timer
    if len(flipped) == 2:
        card1, card2 = flipped
        if card1["icon"] == card2["icon"]:
            card1["matched"] = True
            card2["matched"] = True
            matched.append(card1)
            matched.append(card2)
            flipped = []
        else:
            flip_back_timer = pygame.time.get_ticks() + 1000  # 1 second to memorize the second card

# Flip cards back after the timer
def flip_back():
    global flipped, flip_back_timer
    if flip_back_timer and pygame.time.get_ticks() > flip_back_timer:
        for card in flipped:
            card["flipped"] = False
        flipped = []
        flip_back_timer = 0

# Main game loop
create_cards()
while running:
    screen.fill(WHITE)

    # Event handling
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.MOUSEBUTTONDOWN and event.button == 1:  # Left mouse click
            if not flip_back_timer:  # Prevent new flips during the memorization phase
                pos = pygame.mouse.get_pos()
                for row in cards:
                    for card in row:
                        if card["rect"].collidepoint(pos) and not card["flipped"] and not card["matched"]:
                            card["flipped"] = True
                            flipped.append(card)
                            if len(flipped) == 2:
                                check_match()

    # Flip back unmatched cards
    flip_back()

    # Draw cards
    draw_cards()

    # Check if all cards are matched
    if len(matched) == ROWS * COLS:
        win_text = font.render("You Win!", True, BLACK)
        screen.blit(win_text, (WIDTH // 2 - win_text.get_width() // 2, HEIGHT // 2 - win_text.get_height() // 2))

    pygame.display.update()

pygame.quit()
